In [69]:
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

#模型
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline


import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [70]:
train_url="../rawdata/train.csv"
train_raw=pd.read_csv(train_url)

test_url="../rawdata/test.csv"
test_raw=pd.read_csv(test_url)

In [72]:
#拆分数据集
from sklearn.cross_validation import train_test_split  

#标注正样本和负样本
train_pos=train_raw[train_raw['target']==1]
print (train_pos['target'].value_counts())
train_neg=train_raw[train_raw['target']==0]
print (train_neg['target'].value_counts())

dtypes = train_raw.dtypes
cols = train_raw.columns
ty = dict(zip(cols,dtypes))

#df转narray
train_pos=np.array(train_pos)
train_neg=np.array(train_neg)
#数据集拆分
train_pos_new,validation_pos_new = train_test_split(train_pos,  
                                                   test_size = 0.2,  
                                                   random_state = 0)  
train_neg_new,validation_neg_new = train_test_split(train_neg,  
                                                   test_size = 0.2,  
                                                   random_state = 0)  
#数据集拼接
train_new=np.concatenate([train_pos_new,train_neg_new],axis=0) #在纵轴上合并

validation_new=np.concatenate([validation_pos_new,validation_neg_new],axis=0) #在纵轴上合并


#修改train、validation、test
train_temp=pd.DataFrame(train_new)
train_temp.columns=train_raw.columns
train=train_temp
train = train.astype(ty)

validation_temp=pd.DataFrame(validation_new)
validation_temp.columns=train_raw.columns
validation=validation_temp
validation = validation.astype(ty)

test=test_raw


1    21694
Name: target, dtype: int64
0    573518
Name: target, dtype: int64


In [73]:
train['tag']='train'
validation['tag']='validation'
test['tag']='test'

test['target']=2

print (train.shape)
print (validation.shape)
print (test.shape)

alldata=pd.concat([train,validation,test]) #在纵轴上合并
print (alldata.shape)

(476169, 60)
(119043, 60)
(892816, 60)
(1488028, 60)


In [5]:
#便于调试，做个备份
# tmp_alldata=alldata

# 创建元数据dataframe

In [74]:
data = []
for f in train.columns:
    # Defining the role
    if f == 'target':
        role = 'target'
    elif f == 'id':
        role = 'id'
    elif f == 'tag':
        role = 'tag'
    else:
        role = 'input'
         
    # Defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f in {'id','tag'}:
        level = 'nominal'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f in {'id','tag'}:
        keep = False
    
    # Defining the data type 
    dtype = train[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [75]:
meta

,role,level,keep,dtype
varname,,,,
id,id,nominal,False,int64
target,target,binary,True,int64
ps_ind_01,input,ordinal,True,int64
ps_ind_02_cat,input,nominal,True,int64
ps_ind_03,input,ordinal,True,int64
ps_ind_04_cat,input,nominal,True,int64
ps_ind_05_cat,input,nominal,True,int64
ps_ind_06_bin,input,binary,True,int64
ps_ind_07_bin,input,binary,True,int64


In [76]:
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

,role,level,count
0,id,nominal,1
1,input,binary,17
2,input,interval,10
3,input,nominal,14
4,input,ordinal,16
5,tag,nominal,1
6,target,binary,1


In [77]:
alldata.columns

Index(['id', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin', 'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat',
       'ps_car_11', 'ps_car_11_cat', 'ps_car_12', 'ps_car_13', 'ps_car_14',
       'ps_car_15', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'tag', 'tar

In [78]:
target='target'
IDcol = 'id'
tag='tag'
predictors = [x for x in alldata.columns if x not in [target, IDcol,tag]]

print ('空值占比：')
line_count=alldata['target'].count()
for i in predictors :
    nan_cnt=  line_count - alldata[i][alldata[i] != -1].count()
    if nan_cnt!=0:
        print (i,": {:.5f}% ".format(nan_cnt/(line_count*1.0)*100 ))

空值占比：
ps_car_01_cat : 0.01794% 
ps_car_02_cat : 0.00067% 
ps_car_03_cat : 69.09426% 
ps_car_05_cat : 44.81838% 
ps_car_07_cat : 1.93679% 
ps_car_09_cat : 0.09718% 
ps_car_11 : 0.00040% 
ps_car_12 : 0.00007% 
ps_car_14 : 7.15208% 
ps_ind_02_cat : 0.03515% 
ps_ind_04_cat : 0.01532% 
ps_ind_05_cat : 0.97572% 
ps_reg_03 : 18.10826% 


In [79]:
#剔除类别特征中空置率过高的特征，其他特征保留。保留原因参考（EDA中的分析内容）
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
alldata.drop(vars_to_drop, inplace=True, axis=1)
meta.loc[(vars_to_drop),'keep'] = False  # Updating the meta

In [80]:
#除了类别型特征，存在缺失的特征有 ps_reg_03、ps_car_11、ps_car_12、ps_car_14

#均值填充
mean_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
#最高频率填充
mode_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
#类别特征用-1表示，不做变化

alldata['ps_reg_03'] = mean_imp.fit_transform(alldata[['ps_reg_03']]).ravel()
alldata['ps_car_12'] = mean_imp.fit_transform(alldata[['ps_car_12']]).ravel()
alldata['ps_car_14'] = mean_imp.fit_transform(alldata[['ps_car_14']]).ravel()
alldata['ps_car_11'] = mode_imp.fit_transform(alldata[['ps_car_11']]).ravel()


In [82]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = alldata[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

Variable ps_ind_02_cat has 5 distinct values
Variable ps_ind_04_cat has 3 distinct values
Variable ps_ind_05_cat has 8 distinct values
Variable ps_car_01_cat has 13 distinct values
Variable ps_car_02_cat has 3 distinct values
Variable ps_car_04_cat has 10 distinct values
Variable ps_car_06_cat has 18 distinct values
Variable ps_car_07_cat has 3 distinct values
Variable ps_car_08_cat has 2 distinct values
Variable ps_car_09_cat has 6 distinct values
Variable ps_car_10_cat has 3 distinct values
Variable ps_car_11_cat has 104 distinct values


In [83]:
#ps_car_11_cat类别过多，转为对应target均值的形式

cat_perc = alldata[['ps_car_11_cat', 'target']].groupby(['ps_car_11_cat'],as_index=False).mean()
cat_perc.rename(columns={'target': 'ps_car_11_cat_tm'}, inplace=True)
alldata = pd.merge(alldata, cat_perc, how='inner', on='ps_car_11_cat')
alldata.drop('ps_car_11_cat', axis=1, inplace=True)
meta.loc['ps_car_11_cat','keep'] = False  # Updating the meta

In [88]:
meta[(meta.level == 'nominal') & (meta.keep)].shape

(11, 4)

## 创建哑变量

In [90]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
print('Before dummification we have {} variables in train'.format(alldata.shape[1]))
alldata = pd.get_dummies(alldata, columns=v, drop_first=True)
print('After dummification we have {} variables in train'.format(alldata.shape[1]))

Before dummification we have 110 variables in train
After dummification we have 110 variables in train


In [13]:
meta[(meta.level == 'nominal') & (meta.keep)].shape

(12, 4)

## 特征组合


In [91]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
interactions = pd.DataFrame(data=poly.fit_transform(alldata[v]), columns=poly.get_feature_names(v))
# Merge the interaction variables to the train data
print('Before creating interactions we have {} variables in train'.format(alldata.shape[1]))
alldata = pd.concat([alldata, pd.DataFrame(data=interactions, columns=poly.get_feature_names(v))], axis=1)
print('After creating interactions we have {} variables in train'.format(alldata.shape[1]))

Before creating interactions we have 110 variables in train
After creating interactions we have 175 variables in train


In [92]:
# alldata.id = alldata.id.astype(int)


dtype('int64')

In [93]:
new_train_validation=alldata[ (alldata['tag']=='train') | (alldata['tag']=='validation')]
new_test=alldata[alldata['tag']=='test']
new_train=alldata[ alldata['tag']=='train']
new_validation=alldata[alldata['tag']=='validation']

In [94]:
train_url="../rawdata/train.csv"
train_raw=pd.read_csv(train_url)

new_train_validation.to_csv("../data/train_extends.csv")
new_test.to_csv("../data/test_extends.csv")
alldata.to_csv("../data/all_extends.csv")

In [ ]:
print "Done"